# Experiment

### Imports

In [ ]:
import transform
import numpy as np
from skimage.io import imread
from IPython.display import display, Markdown, Image, HTML
import PIL
from io import BytesIO
from matplotlib import pyplot as plt

### Constants

In [ ]:
DETECTOR_COUNT_MIN = 90
DETECTOR_COUNT_MAX = 720
DETECTOR_COUNT_STEP = 90
DETECTOR_COUNT_DEFAULT = 180

ITERATION_COUNT_MIN = 90
ITERATION_COUNT_MAX = 720
ITERATION_COUNT_STEP = 90
ITERATION_COUNT_DEFAULT = 180

DETECTOR_SPAN_MIN = 45
DETECTOR_SPAN_MAX = 270
DETECTOR_SPAN_STEP = 45
DETECTOR_SPAN_DEFAULT = 180

MAIN_FILE = 'imgs/Shepp_logan.jpg'
SECONDARY_FILE = 'imgs/CT_ScoutView.jpg'

MASK_SIZE = 11

### Functions

In [ ]:
def load_image(filename):
    return transform.normalize_img(imread(filename, as_gray=True))

def display_image(image):
    with BytesIO() as file:
        PIL.Image.fromarray(np.uint8(image * 255)).save(file, format='png')
        display(Image(data=file.getvalue(), width="30%"))

def show_input_image(image):
    display(Markdown("# Input image"))
    display_image(image)

def display_experiment_header(detector_count, iteration_count, detector_span):
    display(Markdown(
        "*** \n"
        f"**Detector count**: {detector_count}<br>"
        f"**Iteration count**: {iteration_count}<br>"
        f"**Detector span**: {detector_span}"))
    
def display_experiment_result(normal, filtered, gaussian_filtered):
    display(Markdown("##### Output: "))
    display_image(normal)
    display(Markdown("##### Convolution: "))
    display_image(filtered)
    display(Markdown("##### Convolution + gaussian: "))
    display_image(gaussian_filtered)
    

def experiment_step(image, offset, r, detector_count = None, iteration_count = None, detector_span = None):
    # setup params
    if detector_count is None:
        detector_count = DETECTOR_COUNT_DEFAULT
    if iteration_count is None:
        iteration_count = ITERATION_COUNT_DEFAULT
    if detector_span is None:
        detector_span = DETECTOR_SPAN_DEFAULT
    
    display_experiment_header(detector_count, iteration_count, detector_span)
    
    emitter_step_rad = np.pi / iteration_count
    detector_span_rad = np.deg2rad(detector_span)
    
    sinogram = transform.img_to_sinogram(image, emitter_step_rad, r, detector_count, detector_span_rad)
    filtered_sinogram = transform.convolve(sinogram, MASK_SIZE)
    
    output_image = transform.sinogram_to_img_simple(sinogram, emitter_step_rad, r, detector_count, detector_span_rad, offset)
    filtered_output_image = transform.sinogram_to_img_simple(filtered_sinogram, emitter_step_rad, r, detector_count, detector_span_rad, offset)
    gaussian_output_image = transform.apply_gaussian(filtered_output_image)
    
    display_experiment_result(output_image, filtered_output_image, gaussian_output_image)
    
    # returns rmse of output, filtered output and filtered output after gaussian
    return [*map(lambda img: transform.root_mean_square_error(input_image, img), 
                      (output_image, filtered_output_image, gaussian_output_image))]

def experiment(image, offset, r, property_name, min_value, max_value, step):
    title = " ".join(property_name.split("_")).capitalize()
    display(Markdown(f"### {title}"))
    display(Markdown(f"##### From {min_value} to {max_value} with step {step}"))
    x = []
    y = []
    for value in range(min_value, max_value + step, step):
        result = experiment_step(image, offset, r, **{property_name: value})
        print(result)
        x.append(value)
        y.append(result)
    y = np.array(y).T
    plt.title(f"Plot of RMSE value against {title.lower()}")
    plt.xlabel(title)
    plt.ylabel("RMSE")
    plt.plot(x, y[0], label="Normal output")
    plt.plot(x, y[1], label="Convolution")
    plt.plot(x, y[2], label="Convolution + gaussian filter")
    plt.show()


### Main

In [ ]:
input_image = load_image(MAIN_FILE)
show_input_image(input_image)

image, offset = transform.resize_to_square(input_image)
r = transform.calc_radius(image)

# uncomment ones to run
# experiment(image, offset, r, "detector_count", DETECTOR_COUNT_MIN, DETECTOR_COUNT_MAX, DETECTOR_COUNT_STEP)
# experiment(image, offset, r, "iteration_count", ITERATION_COUNT_MIN, ITERATION_COUNT_MAX, ITERATION_COUNT_STEP)
experiment(image, offset, r, "detector_span", DETECTOR_SPAN_MIN, DETECTOR_SPAN_MAX, DETECTOR_SPAN_STEP)

display(Markdown("# DONE!!!"))